# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [30]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import os
import gmaps

ModuleNotFoundError: No module named 'gmaps'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
# Study data files
weather_metadata = "C:/Users/iezik/Desktop/USC/usc-la-data-pt-12-2020-u-c/unit_06_python_apis/homework/starter_code/CityWeatherData.csv"

# Read the weather data and the study results
weather_data = pd.read_csv(weather_metadata)

# Combine the data into a single dataset
weatherdf = pd.DataFrame(weather_data)
weatherdf.head()


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Windspeed
0,ostersund,90,SE,1612384661,85,63.1792,14.6357,8.01,5.14
1,ushuaia,75,AR,1612384506,54,-54.8000,-68.3000,53.60,18.41
2,tabuk,75,SA,1612384517,32,28.3833,36.5833,69.80,5.75
3,leningradskiy,65,RU,1612384661,84,69.3833,178.4167,-24.56,11.14
4,bairiki,20,KI,1612384662,83,1.3292,172.9752,80.60,10.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Store 'Lat' and 'Lng' into  locations 
locations = weatherdf[["Lat", "Lng"]].astype(float)

#weatherdf = weatherdf.dropna()

hum_rate = weatherdf["Humidity"].astype(float)

In [8]:
# Create a poverty Heatmap layer
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

fig.add_layer(heat_layer)

plt.savefig("vacationheatmap.png")
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
#Narrow down the DataFrame to find your ideal weather condition.

#A max temperature lower than 80 degrees but higher than 70.
first = weatherdf.loc[weatherdf["Max Temp"]<80,:]
second = first.loc[weatherdf["Max Temp"]>70,:]
second.head()

#Wind speed less than 10 mph.
third = second.loc[second["Windspeed"]<10,:]
third

#Zero cloudiness.
fourth = third.loc[third["Cloudiness"]==0,:]
fourth.head()






,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Windspeed
11,east london,0,ZA,1612384663,81,-33.0153,27.9116,76.68,5.61
15,bandrele,0,YT,1612384494,89,-12.9067,45.1914,78.80,4.61
166,ancud,0,CL,1612384883,56,-41.8697,-73.8203,71.60,6.91
201,castro,0,CL,1612384951,56,-42.4721,-73.7732,71.60,6.91
262,araouane,0,ML,1612385026,19,18.9048,-3.5265,72.75,5.66


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
hotel_df = pd.DataFrame()
fifth = pd.DataFrame(fourth)
hotel_df['City'] = fifth['City']
hotel_df['Country'] = fifth['Country']
hotel_df['Lat'] = fifth['Lat']
hotel_df['Lng'] = fifth['Lng']
#hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Lat,Lng
11,east london,ZA,-33.0153,27.9116
15,bandrele,YT,-12.9067,45.1914
166,ancud,CL,-41.8697,-73.8203
201,castro,CL,-42.4721,-73.7732
262,araouane,ML,18.9048,-3.5265


In [ ]:
#without key
#https://maps.googleapis.com/maps/api/place/textsearch/json?query=hotels&location={longitude},{latitude}&radius=5000&key={g_key}

In [ ]:
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    country = row['Country']

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    
    # extract results
    results = response['results']
    
    try:
        hotelname = response['results'][7]['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

marker_layer = gmaps.marker_layer(locations2, hover_text='', label='', info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
plt.savefig("vacationheatmapwithhotels.png")

fig
